In [28]:
import cv2
import numpy as np
import face_recognition
from imutils import face_utils
import dlib
import os
from datetime import datetime
from scipy.spatial import distance as dist


path = ((r"C:\Users\ARUNASALAM\Desktop\SAMPLE_FACIAL_DATASET"))
images = []
classNames = []
myList = os.listdir(path)
print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

def markAttendance(name):
    with open('Attendence.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

encodeListKnown = findEncodings(images)
print('Encoding Complete')

# p = our pre-treined model directory, on my case, it's on the same script's diretory.
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
 
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
 
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
 
    # return the eye aspect ratio
    return ear

EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 3
 
# initialize the frame counters and the total number of blinks
COUNTER = 0
TOTAL = 0
test = None 

cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    #     img = captureScreen()

    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)
    #eyes related.
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Get faces into webcam's image
    rects = detector(gray, 0)
    
    # For each detected face, find the landmark.
    for (i, rect) in enumerate(rects):
        # Make the prediction and transfom it to numpy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        
        ear = (leftEAR + rightEAR) / 2.0
        
# threshold, and if so, increment the blink frame counter
        if ear < EYE_AR_THRESH:
            COUNTER += 1
            test = False #Mess with this variable to test eye detection********************************
            print('COUNTER:', COUNTER)
    # otherwise, the eye aspect ratio is not below the blink
# threshold
        else:
                # if the eyes were closed for a sufficient number of
                # then increment the total number of blinks
            if COUNTER >= EYE_AR_CONSEC_FRAMES and test == False:
                TOTAL += 1
                print('TOTAL:', TOTAL)
                test = True #Mess with this variable to test eye detection********************************
                # reset the eye frame counter
            #COUNTER = 0
        
            # Draw on our image, all the finded cordinate points (x,y) 
        for (x, y) in shape:
            cv2.circle(img, (x, y), 2, (0, 255, 0), -1)
    
    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        
        # print(faceDis)
        matchIndex = np.argmin(faceDis)
        if faceDis[matchIndex] < 0.50 and test == True:    #50 is too low, raise this
            name = classNames[matchIndex].upper()#RECOGNIZED IN SYSTEM: COLOR IS GREEN
            #markAttendance(name)
            #recognized face    
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
           # print(name) --prints image name
        
        elif faceDis[matchIndex] > 0.50 and test == True:
            name = 'Detecting'#not recognized in the system: LIVE HUMAN DETECTED (NOT RECOGNIZED IN SYSTEM)
            #print('UNKNOWN HUMAN DETECTED') #--prints unknown face
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 255), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 255), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            COUNTER = 0
            test == False #Mess with this variable to test eye detection********************************
        else:
            name = 'Unknown'#not recognized in the system: COLOR IS RED 
            #print("NOT IN THE SYSTEM") #--prints unknown face and no detection of facial liveness
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 0, 255), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            test == False #Mess with this variable to test eye detection********************************
        #recognized face    
        #y1, x2, y2, x1 = faceLoc
        #y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
        #cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        #cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
        #cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow('FACIAL RECOGNITION',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
         break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()    

['Aaron_Eckhart_0001.jpg', 'Aaron_Eckhart_0003.jpg', 'Abdullah_Gul_0001.jpg', 'Abdullah_Gul_0002.jpg', 'Abdullah_Gul_0003.jpg']
Encoding Complete
COUNTER: 1
COUNTER: 2
COUNTER: 3
TOTAL: 1
COUNTER: 1
COUNTER: 2
COUNTER: 3
COUNTER: 4
COUNTER: 5
COUNTER: 6
COUNTER: 7
COUNTER: 8
COUNTER: 9
COUNTER: 10
COUNTER: 11
COUNTER: 12
COUNTER: 13
COUNTER: 14
COUNTER: 15
COUNTER: 16
COUNTER: 17
COUNTER: 18
COUNTER: 19
COUNTER: 20
COUNTER: 21
COUNTER: 22
COUNTER: 23
COUNTER: 24
COUNTER: 25
COUNTER: 26
COUNTER: 27
COUNTER: 28
COUNTER: 29
COUNTER: 30
COUNTER: 31
COUNTER: 32
TOTAL: 2
COUNTER: 33
COUNTER: 34
TOTAL: 3
COUNTER: 35
COUNTER: 36
COUNTER: 37
COUNTER: 38
COUNTER: 39
COUNTER: 40
COUNTER: 41
COUNTER: 42
COUNTER: 43
COUNTER: 44
COUNTER: 45
COUNTER: 46
COUNTER: 47
COUNTER: 48
COUNTER: 49
COUNTER: 50
COUNTER: 51
COUNTER: 52
COUNTER: 53
COUNTER: 54
COUNTER: 55
COUNTER: 56
COUNTER: 57
COUNTER: 58
COUNTER: 59
COUNTER: 60
COUNTER: 61
COUNTER: 62
COUNTER: 63
COUNTER: 64
COUNTER: 65
COUNTER: 66
COUNTER: 67

Note: Face recognition is less effective if facial expressions vary. A big smile can render the system less effective. For instance: Canada, in 2009, allowed only neutral facial expressions in passport photos(Wikipedia).

